In [1]:
import pandas as pd
import numpy as np
import requests
import keys
import json
from tqdm import tqdm

In [72]:
df = pd.read_csv('tm_events.csv')

In [73]:
venues = list(set(df.venue_name))

In [74]:
venues = [x.strip() for x in venues]

In [67]:
url = 'https://api.songkick.com/api/3.0/search/venues.json?'
success = []
fail = []
frame_list = []
for venue in tqdm(venues):
    try:
        req = requests.get(f'{url}query={venue}&apikey={keys.songkick}')
        x = json.loads(req.text)
        df = pd.DataFrame(x['resultsPage']['results']['venue'])
        frame_list.append(df)
        success.append(venue)
    except:
        fail.append(venue)
        


100%|██████████| 2779/2779 [26:50<00:00,  1.61it/s]


In [125]:
df = df[['displayName','id','capacity', 'street', 'city', 'zip', 'lat','lng','metroArea' ]]
df.rename(columns = {'displayName': 'venue_name', 'lat':'latitude', 'lng':'longitude'}, inplace=True)

In [126]:
df.reset_index(inplace=True)
df.rename(columns ={'index' : 'i'}, inplace=True)
df['metro_area'] = df.metroArea.apply(lambda x: x['displayName'])
df.drop(columns = ['city', 'metroArea', 'i'],inplace=True)

In [38]:
df.to_csv('sk_venues.csv')

In [2]:
df = pd.read_csv('sk_venues.csv', dtype='object')

In [3]:
df.drop(columns = ['Unnamed: 0'], inplace=True)

In [4]:
venue_id = df[['venue_name', 'id']]

In [5]:
ids = list(set(venue_id.id))

In [6]:
url = 'https://api.songkick.com/api/3.0/venues/'
success = []
fail = []
frame_list = []
for id_ in tqdm(ids):
    try:
        req = requests.get(f'{url}{str(id_)}/calendar.json?apikey={keys.songkick}')
        x = json.loads(req.text)
        event_df = pd.DataFrame(x['resultsPage']['results']['event'])
        frame_list.append(event_df)
        success.append(id_)
    except:
        fail.append(id_)
        

100%|██████████| 11429/11429 [1:46:07<00:00,  2.19it/s] 


In [7]:
frames = frame_list.copy()

In [9]:
df = pd.concat(frames)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [54]:
df = pd.read_csv('sk_events.csv')

In [55]:
df['location'] = df.location.apply(lambda x: ast.literal_eval(x))
df['venue'] = df.venue.apply(lambda x: ast.literal_eval(x))
df['performance'] = df.performance.apply(lambda x: ast.literal_eval(x))
df['start'] = df.start.apply(lambda x: ast.literal_eval(x))

In [56]:
def try_artist(x, i):
    try:
        return x[i]['displayName']
    except:
        return np.nan
    
df.reset_index(inplace=True)
df.rename(columns={'index' : 'i'},inplace=True)
df['venue_id'] = df.venue.apply(lambda x: x['id'])
df['headliner'] = df.performance.apply(lambda x: try_artist(x, 0))
df['support'] = df.performance.apply(lambda x: try_artist(x, 1))
df['event_time'] = df.start.apply(lambda x: pd.to_datetime(x['datetime']))
df.drop(columns = ['series', 'end','status','uri', 'i','ageRestriction','venue', 'performance', 'location', 'displayName', 'start'], inplace=True)

In [63]:
df.to_csv('sk_events.csv')

In [2]:
venue_df = pd.read_csv('sk_venues.csv')

In [4]:
events_df = pd.read_csv('sk_events.csv')

In [6]:
tm_df = pd.read_csv('tm_events.csv')

In [7]:
artists = list(set(events_df.headliner))

In [8]:
artists = [x for x in artists if x != 'nan']

In [13]:
frame_list = []

In [14]:

url = 'https://api.songkick.com/api/3.0/search/artists.json?'
success = []
fail = []
for artist in tqdm(artists):
    try:
        req = requests.get(f'{url}apikey={keys.songkick}&query={artist}')
        x = json.loads(req.text)
        artist_df = pd.DataFrame(x['resultsPage']['results']['artist'])
        frame_list.append(artist_df)
        success.append(artist)
    except:
        fail.append(artist)

100%|██████████| 11190/11190 [1:26:16<00:00,  2.24it/s]  


In [15]:
frames = frame_list.copy()

In [21]:
df = pd.concat(frames[1:])

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [25]:
df.drop(columns=['identifier','uri'],inplace=True)

In [30]:
df.drop_duplicates(inplace=True)

In [34]:
df.to_csv('artists.csv')

In [44]:
df.head()

,displayName,id,onTourUntil
0,MGMT,137345,2019-11-22
1,T.I.,17879,None
2,P!NK,9862999,2019-09-27
3,M.I.A.,493293,None
4,AWOLNATION,2489666,None


In [53]:
req = requests.get(f'https://api.songkick.com/api/3.0/artists/137345/gigography.json?apikey={keys.songkick}')
x = json.loads(req.text)

In [60]:

url = 'https://api.songkick.com/api/3.0/'
frame_list = []
success = []
fail = []
for id_ in tqdm(ids):
    try:
        req = requests.get(f'{url}artists/{id_}/gigography.json?apikey={keys.songkick}')
        x = json.loads(req.text)
        gig_df = pd.DataFrame(x['resultsPage']['results']['event'])
        frame_list.append(gig_df)
        success.append(id_)
    except:
        fail.append(id_)

100%|██████████| 50839/50839 [6:39:02<00:00,  2.23it/s]    


In [62]:
df = pd.concat(frame_list)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [64]:
df.shape

(897058, 14)

In [66]:
df.to_csv('gigography.csv')

In [2]:
df = pd.read_csv('gigography.csv')

In [3]:
df.shape

(897058, 15)